In [267]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math
import ast
from tqdm import tqdm
import warnings
from pprint import pprint

warnings.filterwarnings("ignore")

In [268]:
all_shots = pd.read_csv("./data/all_shots.csv")

In [269]:
all_shots.shape

(84065, 122)

In [270]:
all_shots = all_shots[~all_shots["competition_id"].isin(["FIFA U20 World Cup", "North American League", "Liga Profesional", "Indian Super League"])].drop(columns=all_shots.columns[all_shots.nunique() == 0]).sort_values(by=["match_id", "timestamp"])
all_shots.nunique()

season_id                   42
match_id                  3312
duration                 77926
id                       83929
index                     4465
location                 56583
minute                     139
off_camera                   1
out                          1
period                       5
play_pattern                 9
player                    5638
player_id                 5637
position                    25
possession                 290
possession_team            288
possession_team_id         291
related_events           83929
second                      60
shot_aerial_won              1
shot_body_part               4
shot_end_location        51584
shot_first_time              1
shot_freeze_frame        82866
shot_key_pass_id         59111
shot_one_on_one              1
shot_outcome                 8
shot_statsbomb_xg        82417
shot_technique               7
shot_type                    5
team                       288
team_id                    291
timestam

In [271]:
all_shots[["season_id", "match_id", "id", "index", "location", "minute", "period", "player", "player_id", "position", "possession", "possession_team", "possession_team_id", "second", "team", "team_id", "timestamp", "competition_id", "shot_outcome"]].sort_values(by=["match_id", "timestamp"])

,season_id,match_id,id,index,location,minute,period,player,player_id,position,possession,possession_team,possession_team_id,second,team,team_id,timestamp,competition_id,shot_outcome
15349,2018/2019,7298.0,9b82eaa3-2048-4157-aa9a-eabeb4fa0ebe,42.0,"[115.0, 25.0]",0.0,1.0,Francesca Kirby,4641.0,Right Center Forward,4.0,Chelsea FCW,971.0,47.0,Chelsea FCW,971.0,00:00:47.620,FA Women's Super League,Blocked
15366,2018/2019,7298.0,62a616ea-6791-422c-af80-1683d22bb9f5,2096.0,"[114.0, 34.0]",47.0,2.0,Ramona Bachmann,4659.0,Right Center Forward,107.0,Chelsea FCW,971.0,59.0,Chelsea FCW,971.0,00:02:59.118,FA Women's Super League,Saved
15367,2018/2019,7298.0,3279e6e0-c796-4871-aaaa-75344dc903b8,2136.0,"[115.0, 41.0]",48.0,2.0,Nikita Parris,4654.0,Right Wing,120.0,Manchester City WFC,746.0,41.0,Manchester City WFC,746.0,00:03:41.638,FA Women's Super League,Goal
15368,2018/2019,7298.0,28c07afb-53b7-497f-9bb0-1c586c27e2de,2154.0,"[109.0, 39.0]",49.0,2.0,Francesca Kirby,4641.0,Left Center Forward,121.0,Chelsea FCW,971.0,37.0,Chelsea FCW,971.0,00:04:37.118,FA Women's Super League,Blocked
15350,2018/2019,7298.0,25dace9c-6bf8-4ada-8a4f-bad0485141c9,237.0,"[109.0, 51.0]",5.0,1.0,Francesca Kirby,4641.0,Right Center Forward,15.0,Chelsea FCW,971.0,12.0,Chelsea FCW,971.0,00:05:12.780,FA Women's Super League,Blocked
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7840,2023,3923881.0,11ac245b-fffb-43b9-a9ab-2953e50e1759,1011.0,"[112.1, 44.3]",37.0,1.0,William Troost-Ekong,5455.0,Center Back,58.0,Nigeria,775.0,1.0,Nigeria,775.0,00:37:01.662,African Cup of Nations,Goal
7851,2023,3923881.0,bded325e-4b96-41d1-992d-3942344b5902,2483.0,"[102.9, 46.8]",89.0,2.0,Kelechi Promise Iheanacho,3708.0,Right Wing,149.0,Nigeria,775.0,12.0,Nigeria,775.0,00:44:12.865,African Cup of Nations,Blocked
7852,2023,3923881.0,3e369ccb-5ae2-45cf-8eb8-f24aec5e9e9d,2623.0,"[97.4, 55.3]",93.0,2.0,Wilfried Stephane Singo,36539.0,Right Back,159.0,Côte d'Ivoire,3374.0,55.0,Côte d'Ivoire,3374.0,00:48:55.619,African Cup of Nations,Wayward
7841,2023,3923881.0,be20d643-eda2-46b4-bd3c-c305e5ce7725,1348.0,"[87.2, 34.1]",49.0,1.0,Seko Fofana,7791.0,Left Center Midfield,82.0,Côte d'Ivoire,3374.0,28.0,Côte d'Ivoire,3374.0,00:49:28.253,African Cup of Nations,Saved


In [272]:
all_passes = pd.read_csv("./data/all_passes.csv")

In [273]:
all_passes

,season_id,match_id,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,goalkeeper_success_in_play,shot_follows_dribble,half_start_late_video_start,player_off_permanent,pass_backheel,goalkeeper_lost_in_play,half_end_early_video_end,goalkeeper_saved_to_post,goalkeeper_penalty_saved_to_post,shot_kick_off
0,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687132,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687133,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687134,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687135,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [274]:
all_shots.nunique()

season_id                   42
match_id                  3312
duration                 77926
id                       83929
index                     4465
location                 56583
minute                     139
off_camera                   1
out                          1
period                       5
play_pattern                 9
player                    5638
player_id                 5637
position                    25
possession                 290
possession_team            288
possession_team_id         291
related_events           83929
second                      60
shot_aerial_won              1
shot_body_part               4
shot_end_location        51584
shot_first_time              1
shot_freeze_frame        82866
shot_key_pass_id         59111
shot_one_on_one              1
shot_outcome                 8
shot_statsbomb_xg        82417
shot_technique               7
shot_type                    5
team                       288
team_id                    291
timestam

In [275]:
all_shots['shot_key_pass_id'].notna().value_counts()

True     59111
False    24818
Name: shot_key_pass_id, dtype: int64

In [276]:
all_shots["related_events"]

15349    ['a1c408ce-f949-4dfd-801f-08ed281da0cc', 'e3be...
15366             ['b5abbcf4-f58b-412c-b55e-7114812fa2f8']
15367             ['4f7c3d8d-a93e-49f5-9c49-c5c29a6f3d86']
15368    ['1a3a5de3-19b3-4ab7-b707-1f4b90c8a5e0', '534f...
15350    ['7d64a307-d6e2-46e7-97cd-211198e0537e', '8bc7...
                               ...                        
7840     ['4708852e-5db3-4c35-be3a-1de2d1304134', '9158...
7851     ['76574587-dc10-4538-be43-aae7d0cfcbe5', 'd4b7...
7852              ['663015a0-57c5-400f-8ced-bf98f9f49ce4']
7841              ['6ecadb13-fc59-4753-9640-df9b5d70b3e8']
7853              ['edb432f3-df04-465b-850e-92ed23209714']
Name: related_events, Length: 83929, dtype: object

In [277]:
# Function to check if each item in the list is in the column
def get_items_in_column(lst, column):
    found_items = []
    for item in lst:
        if item in column.values:
            found_items.append(item)
    return found_items

In [278]:
key_passes = all_passes[all_passes["id"].isin(all_shots['shot_key_pass_id'])]

In [279]:
key_passes

,season_id,match_id,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,goalkeeper_success_in_play,shot_follows_dribble,half_start_late_video_start,player_off_permanent,pass_backheel,goalkeeper_lost_in_play,half_end_early_video_end,goalkeeper_saved_to_post,goalkeeper_penalty_saved_to_post,shot_kick_off
46,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,2015/2016,3890505.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,2015/2016,3890505.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,2015/2016,3890505.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686852,2019,69205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686919,2019,69205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686929,2019,69205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686981,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [280]:
key_passes[["duration", "pass_angle", "pass_type", "pass_height", "pass_length", "pass_assisted_shot_id"]]

,duration,pass_angle,pass_type,pass_height,pass_length,pass_assisted_shot_id
46,1.707827,-1.850475,Corner,High Pass,39.848340,fd99f442-49ae-4724-87e8-2512e4a5d9c6
84,1.635672,1.716809,Corner,High Pass,37.802250,4aaf9747-c33d-4f76-bb0f-d25ad1ebe7c6
234,1.320689,1.778707,Corner,High Pass,37.303352,749a19ff-d599-4170-a095-79b9865e2b06
268,1.350540,-1.871810,Corner,Low Pass,35.077200,37cdb09d-6504-48cb-a6cc-217790d53b4b
307,1.571437,-1.551568,Recovery,High Pass,10.401923,cffe1d92-0692-47ad-a71b-208c8a52dcc0
...,...,...,...,...,...,...
686852,2.271377,-1.125219,Recovery,Ground Pass,22.274874,2c92a25a-2014-4080-b8fd-b69d72092898
686919,1.689300,0.895606,Free Kick,High Pass,34.717430,828295e4-feac-43c4-8b4a-47617f6d2408
686929,1.788300,-1.394087,Free Kick,High Pass,42.664387,58997323-8e35-474d-815c-7926e1e143ba
686981,1.010200,1.630133,Corner,High Pass,30.353418,4201f68c-764d-45bd-bcfb-96470a72c2e8


In [281]:
key_passes["pass_duration"] = key_passes["duration"]

all_shots = pd.merge(all_shots, key_passes[["pass_duration", "pass_angle", "pass_type", "pass_height", "pass_length", "pass_assisted_shot_id"]], how='left', left_on='id', right_on='pass_assisted_shot_id', suffixes=("", ""))
all_shots = all_shots.drop("pass_assisted_shot_id", axis=1)

In [282]:
possession_passes = all_passes.groupby(["match_id", "possession"]).agg({
    'match_id': 'first',
    'possession': 'first',
    'duration': 'count'
}).reset_index(drop=True)
possession_passes["num_passes"] = possession_passes["duration"]
possession_passes = possession_passes.drop("duration", axis=1)

all_shots = pd.merge(all_shots, possession_passes, how='left', on=["match_id", "possession"])

In [283]:
all_shots

,season_id,match_id,duration,id,index,location,minute,off_camera,out,period,...,shot_saved_off_target,shot_saved_to_post,shot_follows_dribble,shot_kick_off,pass_duration,pass_angle,pass_type,pass_height,pass_length,num_passes
0,2018/2019,7298.0,0.560000,9b82eaa3-2048-4157-aa9a-eabeb4fa0ebe,42.0,"[115.0, 25.0]",0.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018/2019,7298.0,0.613000,62a616ea-6791-422c-af80-1683d22bb9f5,2096.0,"[114.0, 34.0]",47.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018/2019,7298.0,0.400000,3279e6e0-c796-4871-aaaa-75344dc903b8,2136.0,"[115.0, 41.0]",48.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018/2019,7298.0,0.120000,28c07afb-53b7-497f-9bb0-1c586c27e2de,2154.0,"[109.0, 39.0]",49.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,1.12,0.927295,Interception,Low Pass,10.0,1.0
4,2018/2019,7298.0,0.400000,25dace9c-6bf8-4ada-8a4f-bad0485141c9,237.0,"[109.0, 51.0]",5.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83924,2023,3923881.0,1.188547,11ac245b-fffb-43b9-a9ab-2953e50e1759,1011.0,"[112.1, 44.3]",37.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
83925,2023,3923881.0,0.314267,bded325e-4b96-41d1-992d-3942344b5902,2483.0,"[102.9, 46.8]",89.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
83926,2023,3923881.0,1.510618,3e369ccb-5ae2-45cf-8eb8-f24aec5e9e9d,2623.0,"[97.4, 55.3]",93.0,NaN,True,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
83927,2023,3923881.0,1.050901,be20d643-eda2-46b4-bd3c-c305e5ce7725,1348.0,"[87.2, 34.1]",49.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [284]:
# extract relevant attributes from the all_shots dataframe
data = pd.DataFrame()

# time attributes
data["period"] = all_shots["period"]
data["minute"] = all_shots["minute"]
data["possession"] = all_shots["possession"]
data["duration"] = all_shots["duration"]
data["season_id"] = all_shots["season_id"]
data["match_id"] = all_shots["match_id"]
data["timestamp"] = all_shots["timestamp"]
data["team"] = all_shots["team"]

data = data.sort_values(by=["match_id", "timestamp"])

In [285]:
# qualitative attributes
data["play_pattern"] = all_shots["play_pattern"]
data["position"] = all_shots["position"]

In [287]:
# shot attributes
data["location_x"] = all_shots["location"].apply(lambda x: ast.literal_eval(x)[0])
data["location_x_distance"] = 120 - data["location_x"]
data["location_y"] = all_shots["location"].apply(lambda x: ast.literal_eval(x)[1])
data["location_y_distance"] = abs(data["location_y"] - 40)
data["duration"] = all_shots["duration"].apply(lambda x: x if x < 100 else 0)
data["technique"] = all_shots["shot_technique"]
data["body_part"] = all_shots["shot_body_part"]
data["type"] = all_shots["shot_type"]
data["is_penalty"] = all_shots["shot_type"] == "Penalty"
data["is_header"] = all_shots["shot_body_part"] == "Head"

In [288]:
# shot modifiers
data["first_time"] = all_shots["shot_first_time"].fillna(False)
data["open_goal"] = all_shots["shot_open_goal"].fillna(False)
data["one_on_one"] = all_shots["shot_one_on_one"].fillna(False)
data["aerial_won"] = all_shots["shot_aerial_won"].fillna(False)
data["follows_dribble"] = all_shots["shot_follows_dribble"].fillna(False)
data["under_pressure"] = all_shots["under_pressure"].fillna(False)

In [289]:
# preceding pass attributes
data["pass_duration"] = all_shots["pass_duration"].fillna(all_shots["pass_duration"].mean())
data["pass_angle"] = all_shots["pass_angle"].fillna(all_shots["pass_angle"].mean())
data["pass_type"] = all_shots["pass_type"].fillna("Missing")
data["pass_height"] = all_shots["pass_height"].fillna("Missing")
data["pass_length"] = all_shots["pass_length"].fillna(all_shots["pass_length"].mean())
data["num_passes"] = all_shots["num_passes"].fillna(0)

In [290]:
# locations for defensive/goalkeeper attributes
locations = pd.DataFrame()
locations["opponents"] = all_shots["shot_freeze_frame"].fillna('[{"location": [], "position": {"name": ""}, "teammate": False}]').apply(lambda frame: [[player["location"], player["position"]["name"]] for player in ast.literal_eval(frame) if not player["teammate"]])
locations["shot"] = tuple(zip(data["location_x"], data["location_y"]))
locations["goalkeeper"] = locations["opponents"].apply(lambda frame: [player[0] for player in frame if player[1] == "Goalkeeper"]).apply(lambda x: x[0] if len(x) != 0 else [-1, -1])
locations

,opponents,shot,goalkeeper
0,"[[[112.0, 28.0], Right Back], [[103.0, 50.0], ...","(115.0, 25.0)","[120.0, 26.0]"
1,"[[[111.0, 41.0], Right Center Back], [[97.0, 4...","(114.0, 34.0)","[119.0, 39.0]"
2,"[[[106.0, 21.0], Right Midfield], [[112.0, 46....","(115.0, 41.0)","[118.0, 40.0]"
3,"[[[109.0, 31.0], Right Center Midfield], [[103...","(109.0, 39.0)","[119.0, 41.0]"
4,"[[[106.0, 42.0], Right Center Midfield], [[115...","(109.0, 51.0)","[119.0, 40.0]"
...,...,...,...
83924,"[[[114.0, 43.2], Right Center Midfield], [[112...","(112.1, 44.3)","[119.0, 41.7]"
83925,"[[[96.3, 37.3], Right Wing], [[106.0, 44.9], R...","(102.9, 46.8)","[116.9, 41.4]"
83926,"[[[117.6, 41.3], Goalkeeper], [[95.6, 56.9], C...","(97.4, 55.3)","[117.6, 41.3]"
83927,"[[[101.3, 20.0], Right Wing Back], [[95.3, 13....","(87.2, 34.1)","[118.5, 39.4]"


In [291]:
def point_in_triangle(point, a, b, c):
    as_x = point[0] - a[0]
    as_y = point[1] - a[1]
    s_ab = ((b[0] - a[0]) * as_y) - ((b[1] - a[1]) * as_x) > 0

    if (((c[0] - a[0]) * as_y) - ((c[1] - a[1]) * as_x) > 0) == s_ab \
            or (((c[0] - b[0]) * (point[1] - b[1])) - ((c[1] - b[1]) * (point[0] - b[0])) > 0) != s_ab:
        return False
    else:
        return True

In [292]:
def angle_between_sides(a, b, c):
    cos_theta = (a**2 + b**2 - c**2) / (2 * a * b)
    theta = math.degrees(math.acos(cos_theta))
    return theta

In [293]:
def best_distance(point) :

    line = (0, 8)
    left_line = (point[0] - 120, point[1] - 36)
    right_line = (point[0] - 120, point[1] - 44)

    line_dot_left = line[0] * left_line[0] + line[1] * left_line[1]
    line_dot_right = line[0] * right_line[0] + line[1] * right_line[1]

    if line_dot_right > 0:
        x = 120 - point[0]
        y = 44 - point[1]
        return math.sqrt(x * x + y * y)
    elif line_dot_left < 0:
        x = 120 - point[0]
        y = 36 - point[1]
        return math.sqrt(x * x + y * y)
    else:
        return 120 - point[0]

In [294]:
# defensive/goalkeeper attributes
data["defenders_3m_radius"] = locations.apply(lambda shot: (len([defender for defender in shot["opponents"] if ((defender[0][0]-shot["shot"][0])**2 + (defender[0][1]-shot["shot"][1])**2) < 3**2])) if shot["opponents"] != [[[], '']] else -1, axis=1)
data["defenders_triangle"] = locations.apply(lambda shot: (len([defender for defender in shot["opponents"] if point_in_triangle(defender[0], shot["shot"], [120, 36], [120, 44])])) if shot["opponents"] != [[[], '']] else -1, axis=1)

data["goalkeeper_x"] = locations["goalkeeper"].apply(lambda x: x[0])
data["goalkeeper_y"] = locations["goalkeeper"].apply(lambda x: x[1])
data["distance_to_goalie"] = data.apply(lambda x: math.sqrt((x["goalkeeper_x"] - x["location_x"])**2 + (x["goalkeeper_y"] - x["location_y"])**2), axis=1)

In [295]:
# angle/location attributes
points = pd.Series(zip(data['location_x'].replace(120.0, 119.9), data['location_y']))
data["shooting_range"] = points.apply(lambda point: angle_between_sides(math.dist(point, (120, 36)), math.dist((120, 44), point), 8))
data["goal_distance"] = points.apply(lambda point: math.dist(point, (120, 40)))
data["best_distance"] = points.apply(lambda point: best_distance(point))
data["goalie_error"] = points.apply(lambda point: )

In [296]:
# target variables
data["statsbomb_xg"] = all_shots["shot_statsbomb_xg"].apply(lambda x: float(x))
data["end_location_x"] = all_shots["shot_end_location"].apply(lambda x: ast.literal_eval(x)[0])
data["end_location_y"] = all_shots["shot_end_location"].apply(lambda x: ast.literal_eval(x)[1])
data["is_goal"] = all_shots["shot_outcome"].apply(lambda x: True if x == "Goal" else False)

In [297]:
data["shot_angle"] =  data.apply(lambda x: math.atan2((x["end_location_y"] - x["location_y"]), (x["end_location_x"] - x["location_x"])), axis=1)
data["good_foot"] = data.apply(lambda shot: True if ((shot["body_part"] == "Right Foot" and shot["location_y"] < 42) or (shot["body_part"] == "Left Foot" and shot["location_y"] > 42)) else False, axis=1)

In [298]:
data["shots_so_far"] = all_shots.groupby(["match_id", "team"])["timestamp"].cumcount()
data["xg_so_far"] = all_shots.groupby(["match_id", "team"])["shot_statsbomb_xg"].cumsum()
data["xg_so_far"] = data["xg_so_far"] - data["statsbomb_xg"]

all_goals = all_shots[all_shots["shot_outcome"] == "Goal"]

first_scorers = all_goals.groupby(['match_id']).first()[["team"]]
all_goals["is_first"] = all_goals[['match_id', 'team']].apply(tuple, axis=1).isin(list(zip(first_scorers.index, first_scorers["team"])))

all_goals["first_tally_temp"] = all_goals[all_goals["is_first"] == True].groupby(["match_id", "is_first"]).cumcount()
all_goals["first_tally"] = all_goals.groupby(["match_id"])["first_tally_temp"].fillna(method="ffill")
all_goals.loc[all_goals['first_tally_temp'].isnull() & ~all_goals['first_tally'].isnull(), 'first_tally'] = all_goals.loc[all_goals['first_tally_temp'].isnull() & ~all_goals['first_tally'].isnull(), 'first_tally'] + 1

all_goals["second_tally_temp"] = all_goals[all_goals["is_first"] == False].groupby(["match_id", "is_first"]).cumcount()
all_goals["second_tally"] = all_goals.groupby(["match_id"])["second_tally_temp"].fillna(method="ffill")
all_goals.loc[all_goals['second_tally_temp'].isnull() & ~all_goals['second_tally'].isnull(), 'second_tally'] = all_goals.loc[all_goals['second_tally_temp'].isnull() & ~all_goals['second_tally'].isnull(), 'second_tally'] + 1
all_goals["second_tally"].fillna(0, inplace=True)

all_goals[["season_id", "match_id", "timestamp", "team", "is_first", "first_tally", "second_tally"]].sort_values(by=["match_id", "timestamp"])

,season_id,match_id,timestamp,team,is_first,first_tally,second_tally
2,2018/2019,7298.0,00:03:41.638,Manchester City WFC,True,0.0,0.0
7,2018/2019,7298.0,00:05:46.380,Chelsea FCW,False,1.0,0.0
13,2018/2019,7298.0,00:23:21.100,Chelsea FCW,False,1.0,1.0
26,2018/2019,7298.0,00:40:34.118,Manchester City WFC,True,1.0,2.0
36,2018,7430.0,00:02:42.533,Washington Spirit,True,0.0,0.0
...,...,...,...,...,...,...,...
83889,2023,3923880.0,00:10:33.248,Congo DR,True,4.0,5.0
83891,2023,3923880.0,00:11:20.504,South Africa,False,5.0,5.0
83915,2023,3923881.0,00:16:40.123,Côte d'Ivoire,True,0.0,0.0
83922,2023,3923881.0,00:35:14.269,Côte d'Ivoire,True,1.0,0.0


In [299]:
data["first_tally"] = all_goals["first_tally"]
data["second_tally"] = all_goals["second_tally"]
data[["season_id", "match_id", "timestamp", "team", "is_goal", "shots_so_far", "xg_so_far", "first_tally", "second_tally"]].sort_values(by=["match_id", "timestamp"])

,season_id,match_id,timestamp,team,is_goal,shots_so_far,xg_so_far,first_tally,second_tally
0,2018/2019,7298.0,00:00:47.620,Chelsea FCW,False,0,0.000000,NaN,NaN
1,2018/2019,7298.0,00:02:59.118,Chelsea FCW,False,1,0.018856,NaN,NaN
2,2018/2019,7298.0,00:03:41.638,Manchester City WFC,True,0,0.000000,0.0,0.0
3,2018/2019,7298.0,00:04:37.118,Chelsea FCW,False,2,0.330081,NaN,NaN
4,2018/2019,7298.0,00:05:12.780,Chelsea FCW,False,3,0.433315,NaN,NaN
...,...,...,...,...,...,...,...,...,...
83924,2023,3923881.0,00:37:01.662,Nigeria,True,2,0.121763,2.0,0.0
83925,2023,3923881.0,00:44:12.865,Nigeria,False,3,0.190122,NaN,NaN
83926,2023,3923881.0,00:48:55.619,Côte d'Ivoire,False,16,1.370430,NaN,NaN
83927,2023,3923881.0,00:49:28.253,Côte d'Ivoire,False,17,1.393774,NaN,NaN


In [300]:
data["is_first"] = data[['match_id', 'team']].apply(tuple, axis=1).isin(list(zip(first_scorers.index, first_scorers["team"])))
data["first_tally"] = data["first_tally"].fillna(method="bfill")
data["second_tally"] = data["second_tally"].fillna(method="bfill")

data["row_index"] = data.index
last_goals = data[data["is_goal"] == True].groupby(["match_id"]).last()["row_index"]
last_shots = data.groupby(["match_id"]).last()["row_index"]
last_shots = last_shots[last_shots.index.isin(last_goals.index)]

for i in range(len(last_goals)):
    if last_goals.iloc[i] != last_shots.iloc[i]:
        goal = data.iloc[last_goals.iloc[i]]
        prop_values = (goal["first_tally"]+1, goal["second_tally"]) if goal["is_first"] else (goal["first_tally"], goal["second_tally"]+1)
        data.loc[last_goals.iloc[i]+1 : last_shots.iloc[i], "first_tally"] = prop_values[0]
        data.loc[last_goals.iloc[i]+1 : last_shots.iloc[i], "second_tally"] = prop_values[1]

data[["season_id", "match_id", "timestamp", "team", "is_goal", "shots_so_far", "xg_so_far", "first_tally", "second_tally"]].sort_values(by=["match_id", "timestamp"])

,season_id,match_id,timestamp,team,is_goal,shots_so_far,xg_so_far,first_tally,second_tally
0,2018/2019,7298.0,00:00:47.620,Chelsea FCW,False,0,0.000000,0.0,0.0
1,2018/2019,7298.0,00:02:59.118,Chelsea FCW,False,1,0.018856,0.0,0.0
2,2018/2019,7298.0,00:03:41.638,Manchester City WFC,True,0,0.000000,0.0,0.0
3,2018/2019,7298.0,00:04:37.118,Chelsea FCW,False,2,0.330081,1.0,0.0
4,2018/2019,7298.0,00:05:12.780,Chelsea FCW,False,3,0.433315,1.0,0.0
...,...,...,...,...,...,...,...,...,...
83924,2023,3923881.0,00:37:01.662,Nigeria,True,2,0.121763,2.0,0.0
83925,2023,3923881.0,00:44:12.865,Nigeria,False,3,0.190122,2.0,1.0
83926,2023,3923881.0,00:48:55.619,Côte d'Ivoire,False,16,1.370430,2.0,1.0
83927,2023,3923881.0,00:49:28.253,Côte d'Ivoire,False,17,1.393774,2.0,1.0


In [301]:
data["game_state"] = data.apply(lambda x: (x["first_tally"] - x["second_tally"]) if x["is_first"] else (x["second_tally"] - x["first_tally"]) ,axis=1)
data["was_leading"] = data["game_state"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)

data.drop(["row_index", "is_first", "first_tally", "second_tally"], axis=1, inplace=True)

data[["season_id", "match_id", "timestamp", "team", "is_goal", "shots_so_far", "xg_so_far", "game_state", "was_leading"]].sort_values(by=["match_id", "timestamp"])

,season_id,match_id,timestamp,team,is_goal,shots_so_far,xg_so_far,game_state,was_leading
0,2018/2019,7298.0,00:00:47.620,Chelsea FCW,False,0,0.000000,0.0,0
1,2018/2019,7298.0,00:02:59.118,Chelsea FCW,False,1,0.018856,0.0,0
2,2018/2019,7298.0,00:03:41.638,Manchester City WFC,True,0,0.000000,0.0,0
3,2018/2019,7298.0,00:04:37.118,Chelsea FCW,False,2,0.330081,-1.0,-1
4,2018/2019,7298.0,00:05:12.780,Chelsea FCW,False,3,0.433315,-1.0,-1
...,...,...,...,...,...,...,...,...,...
83924,2023,3923881.0,00:37:01.662,Nigeria,True,2,0.121763,-2.0,-1
83925,2023,3923881.0,00:44:12.865,Nigeria,False,3,0.190122,-1.0,-1
83926,2023,3923881.0,00:48:55.619,Côte d'Ivoire,False,16,1.370430,1.0,1
83927,2023,3923881.0,00:49:28.253,Côte d'Ivoire,False,17,1.393774,1.0,1


In [302]:
data

,period,minute,possession,duration,season_id,match_id,timestamp,team,play_pattern,position,...,statsbomb_xg,end_location_x,end_location_y,is_goal,shot_angle,good_foot,shots_so_far,xg_so_far,game_state,was_leading
0,1.0,0.0,4.0,0.560000,2018/2019,7298.0,00:00:47.620,Chelsea FCW,From Counter,Right Center Forward,...,0.018856,117.0,34.0,False,1.352127,True,0,0.000000,0.0,0
1,2.0,47.0,107.0,0.613000,2018/2019,7298.0,00:02:59.118,Chelsea FCW,From Counter,Right Center Forward,...,0.311225,119.0,40.9,False,0.943726,False,1,0.018856,0.0,0
2,2.0,48.0,120.0,0.400000,2018/2019,7298.0,00:03:41.638,Manchester City WFC,Regular Play,Right Wing,...,0.259819,120.0,40.7,True,-0.059928,False,0,0.000000,0.0,0
3,2.0,49.0,121.0,0.120000,2018/2019,7298.0,00:04:37.118,Chelsea FCW,Regular Play,Left Center Forward,...,0.103234,110.0,39.0,False,0.000000,True,2,0.330081,-1.0,-1
4,1.0,5.0,15.0,0.400000,2018/2019,7298.0,00:05:12.780,Chelsea FCW,From Throw In,Right Center Forward,...,0.041846,112.0,44.0,False,-1.165905,True,3,0.433315,-1.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83924,1.0,37.0,58.0,1.188547,2023,3923881.0,00:37:01.662,Nigeria,From Corner,Center Back,...,0.068359,120.0,36.9,True,-0.752730,False,2,0.121763,-2.0,-1
83925,2.0,89.0,149.0,0.314267,2023,3923881.0,00:44:12.865,Nigeria,From Free Kick,Right Wing,...,0.052508,104.7,45.9,False,-0.463648,True,3,0.190122,-1.0,-1
83926,2.0,93.0,159.0,1.510618,2023,3923881.0,00:48:55.619,Côte d'Ivoire,From Throw In,Right Back,...,0.023344,117.4,0.1,False,-1.223189,True,16,1.370430,1.0,1
83927,1.0,49.0,82.0,1.050901,2023,3923881.0,00:49:28.253,Côte d'Ivoire,Regular Play,Left Center Midfield,...,0.020162,118.4,38.7,False,0.146381,True,17,1.393774,1.0,1


In [303]:
data.nunique()

period                     5
minute                   139
possession               290
duration               77925
season_id                 42
match_id                3312
timestamp              82726
team                     288
play_pattern               9
position                  25
location_x               635
location_x_distance      635
location_y               701
location_y_distance      498
technique                  7
body_part                  4
type                       5
is_penalty                 2
is_header                  2
first_time                 2
open_goal                  2
one_on_one                 2
aerial_won                 2
follows_dribble            2
under_pressure             2
pass_duration          11897
pass_angle             10504
pass_type                  8
pass_height                4
pass_length             9918
num_passes                13
defenders_3m_radius       10
defenders_triangle        13
goalkeeper_x             258
goalkeeper_y  

In [304]:
data.to_csv("./data/augmented_data.csv", index=False)